In [1]:
# Requests to 4square

In [16]:
import requests
import json
import pandas as pd
from getpass import getpass

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
token = os.getenv("token")

In [19]:
#Connect to MongoDB

In [50]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [20]:
import pymongo
client = pymongo.MongoClient("localhost:27017")
db = client["Ironhack"]
c = db.get_collection("companies")

In [21]:
def requests_for_foursquare (query, lat, lon, radius=500, limit=50):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}&sort=DISTANCE"

    headers = {
        "accept": "application/json",
        "Authorization": token}
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [34]:
def extracting_from_one_element (one_venue):

    name = one_venue["name"]
    distance = one_venue["distance"]
    lat = one_venue["geocodes"]["main"]["latitude"]
    lon = one_venue["geocodes"]["main"]["longitude"]
    
    small_dict = {
    "name": name,
    "distance": distance,
    "location": [lat,lon]
}
    return small_dict

In [35]:
def transform_request_into_df (res):
    list_of_dictionaries = [extracting_from_one_element (element) for element in res["results"]]
    return pd.DataFrame(list_of_dictionaries)

In [36]:
playfish_lat = 51.539778
playfish_long = -0.152998

worldtv_lat = 51.500152 
worldtv_long = -0.126236

pikum_lat = 51.499109 
pikum_long = -0.198480

In [37]:
def request_to_df (venue):
    pikum = requests_for_foursquare (venue, pikum_lat, pikum_long, radius=1000, limit=50)
    df_pikum = transform_request_into_df (pikum)

    worldtv = requests_for_foursquare (venue, worldtv_lat, worldtv_long, radius=1000, limit=50)
    df_worldtv = transform_request_into_df (worldtv)

    playfish = requests_for_foursquare (venue, playfish_lat, playfish_long, radius=1000, limit=50)
    df_playfish = transform_request_into_df (playfish)
    
    merged_df = pd.concat([df_pikum, df_worldtv, df_playfish]).reset_index(drop=True)
    
    return merged_df

In [38]:
def df_to_json (df,file_name):
    json_file = df.to_json(f'data/{file_name}.json', orient='records')
    return json_file

In [39]:
def import_mongo (file_name,collection_name):
    client = pymongo.MongoClient("localhost:27017")
    db = client["Ironhack"]
    c = db.get_collection(f'{collection_name}')
    venues = pd.read_json(f'data/{file_name}.json').to_dict(orient='records')
    c.insert_many(venues)

In [40]:
starbucks = request_to_df ("starbucks")
starbucks_json = df_to_json (starbucks,"london_starbucks")
import_mongo ("london_starbucks","london_starbucks")

In [41]:
schools = request_to_df ("school")
school_json = df_to_json (schools,"london_schools")
import_mongo ("london_schools","london_schools")

In [42]:
party = request_to_df (("bar","club"))
party_json = df_to_json (party,"london_party")
import_mongo ("london_party","london_party")

In [43]:
vegans = request_to_df ("vegan restaurant")
party_json = df_to_json (vegans,"london_vegans")
import_mongo ("london_vegans","london_vegans")

In [44]:
dog_saloons = request_to_df ("dog saloon")
dog_json = df_to_json (dog_saloons,"london_dogsaloons")
import_mongo = ("london_dogsaloons", "longon_dogsaloons")

In [45]:
#Basketball stadium (10 km)

pikum = requests_for_foursquare ("basketball stadium", pikum_lat, pikum_long, radius=10000, limit=10)
df_pikum = transform_request_into_df (pikum)

worldtv = requests_for_foursquare ("basketball stadium", worldtv_lat, worldtv_long, radius=10000, limit=10)
df_worldtv = transform_request_into_df (worldtv)

playfish = requests_for_foursquare ("basketball stadium", playfish_lat, playfish_long, radius=10000, limit=10)
df_playfish = transform_request_into_df (playfish)

basketball_stadiums = pd.concat([df_pikum, df_worldtv, df_playfish]).reset_index(drop=True)

basketball_json = basketball_stadiums.to_json(f'data/basketball_stadium.json', orient='records')

c = db.get_collection(f'london_basketball')
venues = pd.read_json(f'data/basketball_stadium.json').to_dict(orient='records')
c.insert_many(venues)

InsertManyResult([ObjectId('654922b03542e5bc8ead3fa3'), ObjectId('654922b03542e5bc8ead3fa4'), ObjectId('654922b03542e5bc8ead3fa5'), ObjectId('654922b03542e5bc8ead3fa6')], acknowledged=True)

In [46]:
#Working with geojson

In [47]:
"""
new_list = []
for i in london:
    new_dict = {
        "name": i["properties"]["name"],
        "geo": {"type": "Polygon", 
                 "coordinates": i["geometry"]["coordinates"][0]
                }
    }
    new_list.append(new_dict)
new_list
"""

'\nnew_list = []\nfor i in london:\n    new_dict = {\n        "name": i["properties"]["name"],\n        "geo": {"type": "Polygon", \n                 "coordinates": i["geometry"]["coordinates"][0]\n                }\n    }\n    new_list.append(new_dict)\nnew_list\n'

In [48]:
from pymongo import MongoClient

In [49]:
#london is the downloaded geojson (https://github.com/natzar/European-Neighborhoods-Json-Coords/blob/master/london.json)
london[0]
for i in london:
    i['geometry']['coordinates'][0].append(i['geometry']['coordinates'][0][0])

NameError: name 'london' is not defined

In [ ]:
client = MongoClient("localhost:27017")
db = client["Ironhack"]
c = db.get_collection("londonneigh")

for i in london:
    c.insert_one(i)